In [3]:
import numpy as np
from PyLTSpice import SimCommander, RawRead

high_side = True
low_side = False

def frequencytoltpulse(frequency, side: bool):
    deadband = 8  # ns
    period = 1 / (frequency / 1E9)  # Convert to Gigahertz to ensure the period will be in nanoseconds
    if side == low_side:
        return f"PULSE(3.3 0 0n 1n 1n {period + (deadband * 2)}n {period * 2}n)"
    else:
        return f"PULSE(0 3.3 {deadband}n 1n 1n {period}n {period * 2}n)"

fsw = 100E3
L = 10E-6
C = 47E-6

# select spice model
LTC = SimCommander("LTFiles/EPC23102_Mine.asc")
# set default arguments
LTC.set_component_value('VHSin', frequencytoltpulse(fsw, high_side))
LTC.set_component_value('VLSin', frequencytoltpulse(fsw, low_side))

Using LTspice installed in : 'C:\Users\treys\AppData\Local\Programs\ADI\LTspice\LTspice.exe' 
Creating Netlist
The Netlist was successfully created


In [13]:
run_netlist_file = f'LTFiles/F{fsw}_L{L}_C{C}.net'
LTC.run(run_filename=run_netlist_file)

Wed Apr 26 10:40:05 2023 : Starting simulation 2


<RunTask(sim2, started 25412)>

In [14]:
LTR = RawRead(run_netlist_file)
print(LTR.get_trace_names())
vin = LTR.get_trace('V(VVin)')
iin = LTR.get_trace('I(VVin)')
pin = vin * iin

v_ripple = np.ptp()

RuntimeError: Unrecognized encoding